# 01 — Data Collection

This notebook downloads and saves **all raw data sources** needed for the TSLA direction prediction project.

| # | Source | Method |
|---|--------|--------|
| 1 | TSLA daily OHLCV | `yfinance` |
| 2 | Technical indicators | `pandas_ta` |
| 3 | Elon Musk tweets | Kaggle CSV (manual download) |
| 4 | Google Trends | `pytrends` |
| 5 | Fundamentals (revenue, EPS) | `yfinance` |

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from pytrends.request import TrendReq
from src.helpers import DATA_RAW, DATA_PROCESSED, DATA_EXTERNAL, create_target
import time, warnings
warnings.filterwarnings('ignore')

## 1. TSLA Historical OHLCV Data

In [2]:
tsla = yf.download('TSLA', start='2010-01-01', auto_adjust=False)

# Flatten multi-level columns if present
if isinstance(tsla.columns, pd.MultiIndex):
    tsla.columns = tsla.columns.get_level_values(0)

tsla.index = pd.to_datetime(tsla.index).tz_localize(None)
tsla.index.name = 'Date'

# Drop Adj Close (identical to Close with auto_adjust=False after splits)
if 'Adj Close' in tsla.columns:
    tsla.drop(columns=['Adj Close'], inplace=True)

print(f"Shape: {tsla.shape}")
print(f"Date range: {tsla.index.min()} → {tsla.index.max()}")
print(f"Columns: {list(tsla.columns)}")
print(f"\nSanity checks:")
print(f"  Any missing Close? {tsla['Close'].isna().sum()}")
print(f"  Any missing Volume? {tsla['Volume'].isna().sum()}")
print(f"  Any zero Volume? {(tsla['Volume'] == 0).sum()}")
print(f"  Frequency: daily trading days (no weekends/holidays)")
tsla.tail()

[*********************100%***********************]  1 of 1 completed

Shape: (3935, 5)
Date range: 2010-06-29 00:00:00 → 2026-02-19 00:00:00
Columns: ['Close', 'High', 'Low', 'Open', 'Volume']

Sanity checks:
  Any missing Close? 0
  Any missing Volume? 0
  Any zero Volume? 0
  Frequency: daily trading days (no weekends/holidays)


Price,Close,High,Low,Open,Volume
Date,,,,,
2026-02-12,417.070007,436.230011,414.000000,430.299988,61933400
2026-02-13,417.440002,424.059998,410.880005,414.309998,51434100
2026-02-17,410.630005,413.720001,400.510010,412.359985,59678800
2026-02-18,411.320007,416.899994,409.579987,411.109985,45831600
2026-02-19,409.349915,410.299988,404.109985,407.333496,9269167


In [3]:
# Create target variable: 1 if next day's close > today's close
tsla['target'] = create_target(tsla)

print(f"Target distribution:\n{tsla['target'].value_counts(normalize=True).round(3)}")
tsla.to_csv(DATA_RAW / 'tsla_ohlcv.csv')
print(f"Saved to {DATA_RAW / 'tsla_ohlcv.csv'}")

Target distribution:
target
1    0.516
0    0.484
Name: proportion, dtype: float64
Saved to /Users/matheomenges/Desktop/tsla-direction-predictor/data/raw/tsla_ohlcv.csv


## 2. Technical Indicators

In [4]:
df_ta = tsla[['Open', 'High', 'Low', 'Close', 'Volume', 'target']].copy()

# --- Moving Averages ---
df_ta.ta.sma(length=5, append=True)
df_ta.ta.sma(length=10, append=True)
df_ta.ta.sma(length=20, append=True)
df_ta.ta.sma(length=50, append=True)
df_ta.ta.ema(length=12, append=True)
df_ta.ta.ema(length=26, append=True)

# --- RSI ---
df_ta.ta.rsi(length=14, append=True)

# --- MACD ---
df_ta.ta.macd(append=True)

# --- Bollinger Bands ---
df_ta.ta.bbands(length=20, append=True)

# --- ATR (Average True Range) ---
df_ta.ta.atr(length=14, append=True)

# --- Stochastic Oscillator ---
df_ta.ta.stoch(append=True)

# --- OBV (On-Balance Volume) ---
df_ta.ta.obv(append=True)

# --- Historical Volatility (20-day rolling std of returns) ---
df_ta['volatility_20'] = df_ta['Close'].pct_change().rolling(20).std()

print(f"Shape with indicators: {df_ta.shape}")
print(f"New columns: {[c for c in df_ta.columns if c not in tsla.columns]}")
df_ta.to_csv(DATA_RAW / 'tsla_with_ta.csv')
print("Saved.")

Shape with indicators: (3935, 27)
New columns: ['SMA_5', 'SMA_10', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'ATRr_14', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'STOCHh_14_3_3', 'OBV', 'volatility_20']
Saved.


## 3. Elon Musk Tweets

**Manual step:** Download a tweets dataset from Kaggle and place it in `data/external/`.

Recommended datasets (search Kaggle for the most recent):
- *Elon Musk Tweets (2010-2024+)* — look for the largest/most recent one
- Filename assumption: `elonmusk_tweets.csv`

The cell below loads the CSV, removes replies and retweets, and computes a **daily word score** using log-odds differential words — terms statistically overrepresented on days before TSLA up or down moves.  
We use ALL original tweets (not just Tesla-related) — Elon IS Tesla, and his non-Tesla tweets (politics, SpaceX, rants) also predict TSLA direction.

In [5]:
TWEETS_PATH = DATA_EXTERNAL / 'elonmusk_tweets.csv'

if TWEETS_PATH.exists():
    tweets_raw = pd.read_csv(TWEETS_PATH, parse_dates=True)
    print(f"Loaded {len(tweets_raw)} tweets.")
    print(f"Columns: {list(tweets_raw.columns)}")
    tweets_raw.head()
else:
    print(f"⚠ File not found at {TWEETS_PATH}")
    print("Please download a Kaggle tweets dataset and place the CSV there.")
    tweets_raw = None

Loaded 55099 tweets.
Columns: ['id', 'url', 'twitterUrl', 'fullText', 'retweetCount', 'replyCount', 'likeCount', 'quoteCount', 'viewCount', 'createdAt', 'bookmarkCount', 'isReply', 'inReplyToId', 'conversationId', 'inReplyToUserId', 'inReplyToUsername', 'isPinned', 'isRetweet', 'isQuote', 'isConversationControlled', 'possiblySensitive', 'quoteId', 'quote', 'retweet']


In [6]:
import re

# Processing pipeline for raw tweet CSV:
#   Step 1 — Detect date/text columns and drop replies & retweets, keeping only original tweets
#   Step 2 — Clean text (strip URLs, mentions, punctuation) and tokenize with a domain stop-word list
#   Step 3 — Score each tweet using bullish/bearish word sets derived from log-odds ratio analysis
#   Step 4 — Aggregate to daily level: tweet count, word scores, and Tesla-keyword flag

if tweets_raw is not None:
    # Detect the date column
    date_col = None
    for candidate in ['createdAt', 'date', 'created_at', 'datetime', 'timestamp', 'Date']:
        if candidate in tweets_raw.columns:
            date_col = candidate
            break
    
    # Detect the text column
    text_col = None
    for candidate in ['fullText', 'text', 'tweet', 'content', 'Tweet', 'Text']:
        if candidate in tweets_raw.columns:
            text_col = candidate
            break
    
    print(f"Using date_col='{date_col}', text_col='{text_col}'")
    
    # Keep metadata columns
    keep_cols = [date_col, text_col]
    for meta in ['isReply', 'isRetweet', 'isQuote', 'likeCount', 'retweetCount', 'viewCount']:
        if meta in tweets_raw.columns:
            keep_cols.append(meta)
    
    tweets = tweets_raw[keep_cols].copy()
    tweets = tweets.rename(columns={date_col: 'date', text_col: 'text'})
    tweets['date'] = pd.to_datetime(tweets['date'], utc=True, errors='coerce')
    tweets = tweets.dropna(subset=['date', 'text'])
    tweets['date'] = tweets['date'].dt.tz_localize(None).dt.normalize()
    
    # ── STEP 1: Remove replies and retweets ──
    n_before = len(tweets)
    for col in ['isReply', 'isRetweet']:
        if col in tweets.columns:
            tweets[col] = tweets[col].astype(str).str.lower().isin(['true', '1'])
    
    is_reply = tweets.get('isReply', pd.Series(False, index=tweets.index))
    is_retweet = tweets.get('isRetweet', pd.Series(False, index=tweets.index))
    tweets = tweets[~is_reply & ~is_retweet]
    
    print(f"\nTweet type filtering:")
    print(f"  Total tweets: {n_before}")
    print(f"  Replies removed: {is_reply.sum()} ({is_reply.sum()/n_before*100:.1f}%)")
    print(f"  Retweets removed: {is_retweet.sum()} ({is_retweet.sum()/n_before*100:.1f}%)")
    print(f"  Kept (originals): {len(tweets)} ({len(tweets)/n_before*100:.1f}%)")
    
    # ── STEP 2: Clean text and tokenize ──
    def clean_text(text):
        text = re.sub(r'http\S+', '', str(text))
        text = re.sub(r'@\w+', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text.lower().strip()
    
    STOP = set("""
    i me my myself we our ours ourselves you your yours yourself yourselves he him his himself
    she her hers herself it its itself they them their theirs themselves what which who whom this
    that these those am is are was were be been being have has had having do does did doing a an
    the and but if or because as until while of at by for with about against between through during
    before after above below to from up down in out on off over under again further then once here
    there when where why how all both each few more most other some such no nor not only own same
    so than too very s t can will just don should now d ll m o re ve y ain aren couldn didn doesn
    hadn hasn haven isn ma mightn mustn needn shan shouldn wasn weren won wouldn
    also gonna gotta wanna lol yes yeah yep nah true false really actually basically literally
    just like even still well thing things way much many lot lots got get gets going come
    good great best better make makes made right now new already one two three first last next
    back amp https www http co rt thats dont didnt doesnt isnt wasnt wont cant wouldnt couldnt
    youre theyre weve ive hes shes per would could let say said says every need know think
    something anything everything want year years day days time times people take use used
    using im theres heres whats thats gonna need something tell look see goes went doing
    been getting big long high since around tesla teslas elon musk elonmusk
    """.split())
    
    def tokenize(text):
        return [w for w in text.split() if len(w) > 2 and w not in STOP and not w.isdigit()]
    
    tweets['clean'] = tweets['text'].apply(clean_text)
    tweets['tokens'] = tweets['clean'].apply(tokenize)
    
    # ── STEP 3: Word score using log-odds differential words ──
    # These word sets were discovered via log-odds ratio analysis on ALL 14.9k originals
    # Bullish words: overrepresented before TSLA up days (Fisher p < 0.10)
    BULLISH_WORDS = {'air', 'complex', 'details', 'feel', 'highest', 'often', 'sometimes', 'spend'}
    # Bearish words: overrepresented before TSLA down days (Fisher p < 0.10)
    BEARISH_WORDS = {
        'accounts', 'add', 'administration', 'algorithm', 'almost', 'america', 'amount',
        'app', 'away', 'bankrupt', 'biggest', 'bit', 'btw', 'bureaucracy', 'california',
        'call', 'correct', 'crime', 'critical', 'dei', 'dem', 'due', 'electric', 'factory',
        'fast', 'fight', 'fired', 'free', 'government', 'greater', 'happening', 'immediately',
        'improvements', 'increase', 'incredible', 'join', 'law', 'left', 'massive', 'means',
        'meant', 'mind', 'needed', 'nice', 'numbers', 'pennsylvania', 'permanent', 'possible',
        'president', 'press', 'public', 'reach', 'reminder', 'risk', 'seem', 'ship', 'size',
        'smart', 'spending', 'state', 'stop', 'support', 'took', 'trying', 'twitter', 'voice',
        'voters', 'votes', 'within'
    }
    
    tweets['bull_count'] = tweets['tokens'].apply(lambda t: len(set(t) & BULLISH_WORDS))
    tweets['bear_count'] = tweets['tokens'].apply(lambda t: len(set(t) & BEARISH_WORDS))
    tweets['word_score'] = tweets['bull_count'] - tweets['bear_count']
    
    # ── STEP 4: Daily aggregation ──
    tweets_daily = tweets.groupby('date').agg(
        tweet_count=('text', 'count'),
        tweet_bull_count=('bull_count', 'sum'),
        tweet_bear_count=('bear_count', 'sum'),
        tweet_word_score=('word_score', 'sum'),
    )
    tweets_daily.index = pd.to_datetime(tweets_daily.index)
    
    # Also flag Tesla-specific tweet days
    TESLA_KEYWORDS = ['tesla', 'tsla', 'fsd', 'cybertruck', 'robotaxi',
                      'deliveries', 'model y', 'model 3', 'model s', 'model x',
                      'autopilot', 'gigafactory', 'supercharger', 'megapack',
                      'powerwall', 'full self driving', 'self driving']
    text_lower = tweets['text'].str.lower()
    tesla_mask = pd.Series(False, index=tweets.index)
    for kw in TESLA_KEYWORDS:
        tesla_mask |= text_lower.str.contains(kw, na=False)
    
    tesla_daily = tweets[tesla_mask].groupby('date').agg(
        tesla_tweet_count=('text', 'count'),
    )
    tweets_daily = tweets_daily.join(tesla_daily, how='left')
    tweets_daily['tesla_tweet_count'] = tweets_daily['tesla_tweet_count'].fillna(0).astype(int)
    tweets_daily['has_tesla_tweet'] = (tweets_daily['tesla_tweet_count'] > 0).astype(int)
    
    print(f"\nDaily aggregated shape: {tweets_daily.shape}")
    print(f"Days with any tweet: {len(tweets_daily)}")
    print(f"Days with Tesla tweet: {int(tweets_daily['has_tesla_tweet'].sum())}")
    print(f"\nWord score stats:")
    print(tweets_daily['tweet_word_score'].describe().round(2))
    print(f"\nColumns: {list(tweets_daily.columns)}")
    
    tweets_daily.to_csv(DATA_RAW / 'elon_tweets_daily.csv')
    print(f"\nSaved to {DATA_RAW / 'elon_tweets_daily.csv'}")
    tweets_daily.tail()
else:
    print("Skipping tweet processing — no data loaded.")

Using date_col='createdAt', text_col='fullText'

Tweet type filtering:
  Total tweets: 55099
  Replies removed: 39096 (71.0%)
  Retweets removed: 1092 (2.0%)
  Kept (originals): 14911 (27.1%)

Daily aggregated shape: (2911, 6)
Days with any tweet: 2911
Days with Tesla tweet: 950

Word score stats:
count    2911.0
mean       -1.3
std         3.2
min       -43.0
25%        -1.0
50%         0.0
75%         0.0
max         2.0
Name: tweet_word_score, dtype: float64

Columns: ['tweet_count', 'tweet_bull_count', 'tweet_bear_count', 'tweet_word_score', 'tesla_tweet_count', 'has_tesla_tweet']

Saved to /Users/matheomenges/Desktop/tsla-direction-predictor/data/raw/elon_tweets_daily.csv


## 4. Google Trends (Categorized, Weekly)

We use pre-collected weekly Google Trends data with **actionable, categorized** search terms:

| Category | Terms | Signal |
|----------|-------|--------|
| **risk** | tesla recall, crash, lawsuit, investigation | Spikes during negative events |
| **investor** | buy/sell tesla stock, tsla earnings, stock price | Active trading interest |
| **product** | tesla fsd, robotaxi, cybertruck delivery, model 2 | Product catalyst attention |
| **brand** | Tesla (baseline) | Overall brand awareness |

Each category is collected separately so niche terms aren't crushed to 0 by the dominant "Tesla" baseline.
Overlapping 5-year windows give **weekly** granularity (vs monthly for generic pytrends queries).

In [7]:
# Load pre-collected granular Google Trends data
# (collected by data/google_trends_collection.py with categorized search terms)
GTRENDS_PATH = DATA_RAW / 'google_trends_granular.csv'

if GTRENDS_PATH.exists():
    gtrends = pd.read_csv(GTRENDS_PATH, index_col='Date', parse_dates=True)
    gtrends.index = gtrends.index.tz_localize(None)
    print(f"Google Trends shape: {gtrends.shape}")
    print(f"Date range: {gtrends.index.min().date()} → {gtrends.index.max().date()}")
    print(f"Granularity: weekly ({len(gtrends)} data points)")
    print(f"\nColumns by category:")
    for cat in ['tesla', 'risk', 'investor', 'product']:
        cols = [c for c in gtrends.columns if f'gtrend_{cat}' in c]
        if cols:
            print(f"  {cat}: {cols}")
    print(f"\nSample (last 3 rows):")
    print(gtrends.tail(3).to_string())
else:
    print(f"⚠ Google Trends file not found at {GTRENDS_PATH}")
    print("Run: python data/google_trends_collection.py")
    gtrends = None

Google Trends shape: (328, 13)
Date range: 2016-01-01 → 2026-02-15
Granularity: weekly (328 data points)

Columns by category:
  tesla: ['gtrend_tesla']
  risk: ['gtrend_risk_tesla_recall', 'gtrend_risk_tesla_crash', 'gtrend_risk_tesla_lawsuit', 'gtrend_risk_tesla_investigation']
  investor: ['gtrend_investor_buy_tesla_stock', 'gtrend_investor_sell_tesla_stock', 'gtrend_investor_tsla_earnings', 'gtrend_investor_tesla_stock_price']
  product: ['gtrend_product_tesla_fsd', 'gtrend_product_tesla_robotaxi', 'gtrend_product_cybertruck_delivery', 'gtrend_product_tesla_model_2']

Sample (last 3 rows):
            gtrend_tesla  gtrend_risk_tesla_recall  gtrend_risk_tesla_crash  gtrend_risk_tesla_lawsuit  gtrend_risk_tesla_investigation  gtrend_investor_buy_tesla_stock  gtrend_investor_sell_tesla_stock  gtrend_investor_tsla_earnings  gtrend_investor_tesla_stock_price  gtrend_product_tesla_fsd  gtrend_product_tesla_robotaxi  gtrend_product_cybertruck_delivery  gtrend_product_tesla_model_2
Date   

## 5. Fundamentals (Quarterly Revenue & EPS)

In [8]:
# Load comprehensive fundamentals from curated CSV (SEC filings data, 2012-2025)
# yfinance only returns ~5 recent quarters, so we use a manually compiled dataset

FUND_PATH = DATA_EXTERNAL / 'tsla_quarterly_fundamentals.csv'
fundamentals = pd.read_csv(FUND_PATH, parse_dates=['quarter_end'], index_col='quarter_end')
fundamentals.index.name = 'Date'

# Add derived features
fundamentals['revenue_growth_qoq'] = fundamentals['quarterly_revenue'].pct_change()
fundamentals['eps_growth_qoq'] = fundamentals['eps_diluted'].diff()
fundamentals['profit_margin'] = fundamentals['net_income'] / fundamentals['quarterly_revenue']

print(f"Fundamentals: {fundamentals.shape[0]} quarters ({fundamentals.index.min().date()} → {fundamentals.index.max().date()})")
print(f"Columns: {list(fundamentals.columns)}")
print(f"\nMissing values:\n{fundamentals.isna().sum()}")

fundamentals.to_csv(DATA_RAW / 'tsla_fundamentals.csv')
print(f"\nSaved to {DATA_RAW / 'tsla_fundamentals.csv'}")
fundamentals.tail(10)

Fundamentals: 56 quarters (2012-03-31 → 2025-12-31)
Columns: ['quarterly_revenue', 'eps_diluted', 'net_income', 'revenue_growth_qoq', 'eps_growth_qoq', 'profit_margin']

Missing values:
quarterly_revenue     0
eps_diluted           0
net_income            0
revenue_growth_qoq    1
eps_growth_qoq        1
profit_margin         0
dtype: int64

Saved to /Users/matheomenges/Desktop/tsla-direction-predictor/data/raw/tsla_fundamentals.csv


,quarterly_revenue,eps_diluted,net_income,revenue_growth_qoq,eps_growth_qoq,profit_margin
Date,,,,,,
2023-09-30,23350000000,0.53,1853000000,-0.063265,-0.25,0.079358
2023-12-31,25167000000,2.27,7928000000,0.077816,1.74,0.315016
2024-03-31,21301000000,0.41,1129000000,-0.153614,-1.86,0.053002
2024-06-30,25500000000,0.40,1478000000,0.197127,-0.01,0.057961
2024-09-30,25182000000,0.62,2167000000,-0.012471,0.22,0.086054
2024-12-31,25707000000,0.61,2314000000,0.020848,-0.01,0.090014
2025-03-31,19335000000,0.12,409000000,-0.247870,-0.49,0.021153
2025-06-30,22496000000,0.33,1134000000,0.163486,0.21,0.050409
2025-09-30,28095000000,0.39,1853000000,0.248889,0.06,0.065955


## 6. Merge Everything into One Master Dataset

All external data is aligned to TSLA trading days and forward-filled where appropriate.

In [9]:
from src.helpers import align_to_trading_days

master = df_ta.copy()
trading_dates = master.index

# --- Merge tweets ---
if TWEETS_PATH.exists():
    tw = align_to_trading_days(tweets_daily, trading_dates)
    # Fill days with no tweets as 0
    for c in tw.columns:
        tw[c] = tw[c].fillna(0)
    master = master.join(tw, how='left')

# --- Merge Google Trends (weekly → forward-fill to daily) ---
if gtrends is not None:
    gt = align_to_trading_days(gtrends, trading_dates)
    master = master.join(gt, how='left')

# --- Merge fundamentals (quarterly → forward-fill) ---
if not fundamentals.empty:
    fund = align_to_trading_days(fundamentals, trading_dates)
    master = master.join(fund, how='left')

print(f"Master dataset shape: {master.shape}")
print(f"Columns ({len(master.columns)}): {list(master.columns)}")
master.to_csv(DATA_PROCESSED / 'master_dataset.csv')
print(f"Saved to {DATA_PROCESSED / 'master_dataset.csv'}")
master.tail()

Master dataset shape: (3935, 52)
Columns (52): ['Open', 'High', 'Low', 'Close', 'Volume', 'target', 'SMA_5', 'SMA_10', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26', 'RSI_14', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0_2.0', 'BBM_20_2.0_2.0', 'BBU_20_2.0_2.0', 'BBB_20_2.0_2.0', 'BBP_20_2.0_2.0', 'ATRr_14', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'STOCHh_14_3_3', 'OBV', 'volatility_20', 'tweet_count', 'tweet_bull_count', 'tweet_bear_count', 'tweet_word_score', 'tesla_tweet_count', 'has_tesla_tweet', 'gtrend_tesla', 'gtrend_risk_tesla_recall', 'gtrend_risk_tesla_crash', 'gtrend_risk_tesla_lawsuit', 'gtrend_risk_tesla_investigation', 'gtrend_investor_buy_tesla_stock', 'gtrend_investor_sell_tesla_stock', 'gtrend_investor_tsla_earnings', 'gtrend_investor_tesla_stock_price', 'gtrend_product_tesla_fsd', 'gtrend_product_tesla_robotaxi', 'gtrend_product_cybertruck_delivery', 'gtrend_product_tesla_model_2', 'quarterly_revenue', 'eps_diluted', 'net_income', 'revenue_growth_qoq', 'eps_growth

,Open,High,Low,Close,Volume,target,SMA_5,SMA_10,SMA_20,SMA_50,...,gtrend_product_tesla_fsd,gtrend_product_tesla_robotaxi,gtrend_product_cybertruck_delivery,gtrend_product_tesla_model_2,quarterly_revenue,eps_diluted,net_income,revenue_growth_qoq,eps_growth_qoq,profit_margin
Date,,,,,,,,,,,,,,,,,,,,,
2026-02-12,430.299988,436.230011,414.000000,417.070007,61933400,1,419.795996,417.637997,425.783998,444.184200,...,16.0,2.0,0.0,8.0,2.490100e+10,0.24,1.076000e+09,-0.113686,-0.15,0.043211
2026-02-13,414.309998,424.059998,410.880005,417.440002,51434100,0,421.062000,416.340997,424.727498,443.948201,...,16.0,2.0,0.0,8.0,2.490100e+10,0.24,1.076000e+09,-0.113686,-0.15,0.043211
2026-02-17,412.359985,413.720001,400.510010,410.630005,59678800,1,419.723999,415.222998,423.383998,443.226001,...,17.0,3.0,0.0,8.0,2.490100e+10,0.24,1.076000e+09,-0.113686,-0.15,0.043211
2026-02-18,411.109985,416.899994,409.579987,411.320007,45831600,0,416.946002,414.159000,422.987498,442.361801,...,17.0,3.0,0.0,8.0,2.490100e+10,0.24,1.076000e+09,-0.113686,-0.15,0.043211
2026-02-19,407.333496,410.299988,404.109985,409.349915,9269167,0,413.161987,414.492990,421.882994,441.448799,...,17.0,3.0,0.0,8.0,2.490100e+10,0.24,1.076000e+09,-0.113686,-0.15,0.043211


In [10]:
from src.helpers import missing_report
missing_report(master)

,missing,pct
gtrend_product_tesla_model_2,1388,35.27
gtrend_risk_tesla_lawsuit,1388,35.27
gtrend_risk_tesla_recall,1388,35.27
gtrend_tesla,1388,35.27
gtrend_risk_tesla_investigation,1388,35.27
gtrend_investor_buy_tesla_stock,1388,35.27
gtrend_investor_sell_tesla_stock,1388,35.27
gtrend_investor_tsla_earnings,1388,35.27
gtrend_investor_tesla_stock_price,1388,35.27
gtrend_product_tesla_fsd,1388,35.27


## 7. Data Quality Summary

In [11]:
# ── Data quality overview ──
print("=" * 60)
print("DATA QUALITY SUMMARY")
print("=" * 60)

print(f"\nMASTER DATASET: {master.shape[0]} trading days x {master.shape[1]} columns")
print(f"   Date range: {master.index.min().date()} -> {master.index.max().date()}")
print(f"   Target: {master['target'].value_counts()[1]} up days ({master['target'].mean():.1%}), "
      f"{master['target'].value_counts()[0]} down days")

print(f"\nOHLCV: complete ({tsla.isna().sum().sum()} missing values)")
print(f"   Volume range: {tsla['Volume'].min():,.0f} -- {tsla['Volume'].max():,.0f}")

print(f"\nTECHNICAL INDICATORS: {len([c for c in df_ta.columns if c not in tsla.columns])} features")
print(f"   Warmup NaN rows: ~{df_ta[['SMA_50']].isna().sum().values[0]} (first {df_ta[['SMA_50']].isna().sum().values[0]} days)")

if TWEETS_PATH.exists():
    tesla_tweet_days = tweets_daily['has_tesla_tweet'].sum()
    print(f"\nTWEETS: {len(tweets_raw)} raw -> {len(tweets_daily)} daily aggregates")
    print(f"   Date range: {tweets_daily.index.min().date()} -> {tweets_daily.index.max().date()}")
    print(f"   Days with Tesla-related keywords: {tesla_tweet_days} ({tesla_tweet_days/len(tweets_daily):.1%})")
    print(f"   Note: Sparse before 2017 -- most tweets are general, not Tesla-specific")

if gtrends is not None:
    gt_cats = {}
    for c in gtrends.columns:
        parts = c.replace('gtrend_', '').split('_', 1)
        cat = parts[0] if len(parts) > 1 else 'brand'
        gt_cats.setdefault(cat, []).append(c)
    print(f"\nGOOGLE TRENDS: {len(gtrends)} weekly data points ({gtrends.index.min().date()} -> {gtrends.index.max().date()})")
    print(f"   {len(gtrends.columns)} signals across {len(gt_cats)} categories: {list(gt_cats.keys())}")
    print(f"   Granularity: weekly (forward-filled to daily)")
    print(f"   Note: Missing for 2010-2015 (pre-collection range)")

if not fundamentals.empty:
    print(f"\nFUNDAMENTALS: {len(fundamentals)} quarters")
    print(f"   Date range: {fundamentals.index.min().date()} -> {fundamentals.index.max().date()}")
    print(f"   Columns: {list(fundamentals.columns)}")

print("\n" + "=" * 60)
print("RECOMMENDATION FOR MODELING:")
print("=" * 60)
print("* Strong features: OHLCV + all technical indicators (complete)")
print("* Moderate: Tweet sentiment (useful from ~2017 onward)")
print("* Moderate: Google Trends (weekly categorized -- risk/investor/product signals)")
print("* Moderate: Fundamentals (quarterly, forward-filled)")
print("* Consider training on 2016+ data for best feature coverage")

DATA QUALITY SUMMARY

MASTER DATASET: 3935 trading days x 52 columns
   Date range: 2010-06-29 -> 2026-02-19
   Target: 2029 up days (51.6%), 1906 down days

OHLCV: complete (0 missing values)
   Volume range: 1,777,500 -- 914,082,000

TECHNICAL INDICATORS: 21 features
   Warmup NaN rows: ~49 (first 49 days)

TWEETS: 55099 raw -> 2911 daily aggregates
   Date range: 2010-06-04 -> 2025-04-13
   Days with Tesla-related keywords: 950 (32.6%)
   Note: Sparse before 2017 -- most tweets are general, not Tesla-specific

GOOGLE TRENDS: 328 weekly data points (2016-01-01 -> 2026-02-15)
   13 signals across 4 categories: ['brand', 'risk', 'investor', 'product']
   Granularity: weekly (forward-filled to daily)
   Note: Missing for 2010-2015 (pre-collection range)

FUNDAMENTALS: 56 quarters
   Date range: 2012-03-31 -> 2025-12-31
   Columns: ['quarterly_revenue', 'eps_diluted', 'net_income', 'revenue_growth_qoq', 'eps_growth_qoq', 'profit_margin']

RECOMMENDATION FOR MODELING:
* Strong features: O